# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from implicit.als import AlternatingLeastSquares
import scipy.sparse as sparse
from sklearn.model_selection import train_test_split
import json

/Users/jordan/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# boto3 for S3 access
import boto3

# Getting AWS Book Reviews Data Subset

This code can be used to load in the AWS JSONL review and book details datasets. They are quite large (about 30 GB combined), so it only loads a small subset into memory. I had trouble working with the whole dataset even on the largest compute available for the lab (r6i-large). So I used a different dataset below to get the model working and plan to return to this one or a previous year with less reviews to build out a complete model. 

In [ ]:
# Define the file path
file_path = "../../data/raw/dataset/Books.jsonl"

# Maximum number of lines to read (smaller dataset)
max_lines = 10000

# Read the file line by line and parse each line as a JSON object
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        if len(data) >= max_lines:
            break
        try:
            # Parse the JSON string from the line
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            # You can add logic here to log or skip malformed lines
            print(f"Skipping malformed line: {e} - Line content: {line[:50]}...") 
            continue

# Convert the list of dictionaries into a pandas DataFrame
books = pd.DataFrame(data)

books.head()

   rating                                              title  \
0     1.0      Not a watercolor book! Seems like copies imo.   
1     5.0  Updated: after 1st arrived damaged this one is...   
2     5.0                              Excellent! I love it!   
3     5.0       Updated after 1st arrived damaged. Excellent   
4     5.0                                Beautiful patterns!   

                                                text  \
0  It is definitely not a watercolor book.  The p...   
1  Updated: after first book arrived very damaged...   
2  I bought it for the bag on the front so it pai...   
3  Updated: after 1st arrived damaged the replace...   
4  I love this book!  The patterns are lovely. I ...   

                                              images        asin parent_asin  \
0  [{'small_image_url': 'https://m.media-amazon.c...  B09BGPFTDB  B09BGPFTDB   
1                                                 []  0593235657  0593235657   
2                                     

In [ ]:
import pandas as pd
import json

# Define the file path
file_path = "../../data/raw/dataset/meta_Books.jsonl"

# Maximum number of lines to read
max_lines = 1000000

# Read the file line by line and parse each line as a JSON object
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        if len(data) >= max_lines:
            break
        try:
            # Parse the JSON string from the line
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            # You can add logic here to log or skip malformed lines
            print(f"Skipping malformed line: {e} - Line content: {line[:50]}...") 
            continue

# Convert the list of dictionaries into a pandas DataFrame
meta = pd.DataFrame(data)

meta.head()

  main_category                                              title  \
0         Books                                            Chaucer   
1         Books                            Notes from a Kidwatcher   
2         Books                        Service: A Navy SEAL at War   
3         Books  Monstrous Stories #4: The Day the Mice Stood S...   
4  Buy a Kindle                                    Parker & Knight   

                              subtitle  \
0  Hardcover – Import, January 1, 2004   
1                        First Edition   
2              Hardcover – May 8, 2012   
3         Paperback – October 29, 2013   
4                       Kindle Edition   

                                              author  average_rating  \
0  {'avatar': 'https://m.media-amazon.com/images/...             4.5   
1  {'avatar': 'https://m.media-amazon.com/images/...             5.0   
2  {'avatar': 'https://m.media-amazon.com/images/...             4.7   
3                                     

In [45]:
# Find out how the two dataframes can be joined
# Find all the rows where parent_asin in meta matches parent_asin in books
# Use an inner merge on the shared column
matching_rows = books.merge(meta, on="parent_asin", how="inner", suffixes=('_review', '_book'))

# Count the number of matches
num_matches = len(matching_rows)
print(f"Number of rows with matching 'parent_asin' values: {num_matches}")

Number of rows with matching 'parent_asin' values: 3016


That's about 30% of the data we loaded in, so I think that the columns are probably the same, since the reviews data might not be in the same order as the books (meaning books in the later part of the data wouldn't be found to match the parent_asin of the review). I decided to proceed with this further limited data to test the algorithm, and train on a larger portion later.

In [46]:
matching_rows.head()

,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,rating_number,features,description,price,images_book,videos,store,categories,details,bought_together
0,5.0,Excellent! I love it!,I bought it for the bag on the front so it pai...,[],1782490671,1782490671,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1640383495102,0,True,...,306,[Make 35 gorgeous crochet projects all incorpo...,"[Book Description, Make 35 gorgeous crochet pr...",21.95,[{'large': 'https://m.media-amazon.com/images/...,[],Nicki Trench (Author),"[Books, Crafts, Hobbies & Home, Crafts & Hobbies]",{'Publisher': 'CICO Books; US edition (Septemb...,None
1,5.0,Beautiful patterns!,I love this book! The patterns are lovely. I ...,[{'small_image_url': 'https://m.media-amazon.c...,0823098079,0823098079,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1637312253230,0,True,...,68,"[Haiku, the graceful and evocative form of Jap...","[About the Author, TANYA ALPERT is a knitwear ...",19.61,[{'large': 'https://m.media-amazon.com/images/...,[],Tanya Alpert (Author),"[Books, Crafts, Hobbies & Home, Crafts & Hobbies]","{'Publisher': 'Potter Craft (October 20, 2009)...",None
2,3.0,"Half the size of her other books, but same pri...",I really wanted to like this book bc I have he...,[{'small_image_url': 'https://images-na.ssl-im...,1645671127,1645671127,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1612044209266,3,False,...,587,[Capture the Vibrant Colors of the Jungle with...,"[From the Author, ""Watercolor With Me: In The ...",15.89,[{'large': 'https://m.media-amazon.com/images/...,"[{'title': 'Fewer and Smaller Projects', 'url'...",Dana Fox (Author),"[Books, Arts & Photography, History & Criticism]",{'Publisher': 'Page Street Publishing (October...,None
3,1.0,Crease down entire side of every page!!!,Every page has a crease running the entire len...,[{'small_image_url': 'https://images-na.ssl-im...,1780671067,1780671067,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1611623223325,2,True,...,15393,[Secret Garden: An Inky Treasure Hunt and Colo...,"[Review, ""...a coloring book even adults will ...",12.15,[{'large': 'https://m.media-amazon.com/images/...,[{'title': 'Secret Garden - Inky Treasure Hunt...,Johanna Basford (Author),"[Books, Arts & Photography, Graphic Design]",{'Publisher': 'Laurence King Publishing; Act C...,None
4,5.0,Granddaughter loves it!,This was my 6th copy! All 5 of my kids had th...,[],1442450703,1442450703,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1523093714024,94,True,...,36580,[There is always enough room on your child’s b...,"[About the Author, Bill Martin, Jr. (1916–2004...",4.59,[{'large': 'https://m.media-amazon.com/images/...,[{'title': 'One of our children's all-time fav...,"Bill Martin Jr. (Author), John Archambault (A...","[Books, Children's Books, Literature & Fiction]","{'Publisher': 'Little Simon (January 1, 2012)'...",None


In [ ]:
# Need to make a table with user-item interactions
# The table should have users as rows and items as columns (or vice versa)
user_item_matrix = sparse.csr_matrix((books['rating'], (books['user_id'].astype('category').cat.codes, books['title'].astype('category').cat.codes)))

In [ ]:
# Creating ALS model for recommendation.
model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
# Fitting to user item matrix. 
model.fit(user_item_matrix.T)

To test the trained model, I created a new user with random 5 star reviews (book 0 and book 10). 

In [ ]:
# Create the Categorical Series first
title_category = matching_rows["title_book"].astype("category")

title_to_index = dict(zip(title_category.cat.categories, title_category.cat.codes))

index_to_title = dict(enumerate(title_category.cat.categories))

In [35]:
u_new = np.zeros((1, user_item_matrix.shape[1]))
u_new[0, 10] = 5  # Assume the new user rated item with index 10 a 5
u_new_sparse = sparse.csr_matrix(u_new)
recommended_new = model.recommend(0, u_new_sparse, N=10)
print(f"Recommendations for new user: {recommended_new}")

Recommendations for new user: (array([2495, 1374, 1364, 1233, 1084, 1010,  625,  442,  391,  248],
      dtype=int32), array([0.21358046, 0.18483542, 0.18483542, 0.18483542, 0.18483542,
       0.18483542, 0.18483542, 0.18483542, 0.18483542, 0.18483542],
      dtype=float32))


In [ ]:
u_new = np.zeros((1, user_item_matrix.shape[1]))
u_new[0, 10] = 5  # Assume the new user rated item with index 10 a 5
recommended = sorted(np.dot(u_new, model.item_factors))[:10]
print(f"Recommendations for new user: {recommended_new}")

Recommendations for new user: (array([2495, 1374, 1364, 1233, 1084, 1010,  625,  442,  391,  248],
      dtype=int32), array([0.21358046, 0.18483542, 0.18483542, 0.18483542, 0.18483542,
       0.18483542, 0.18483542, 0.18483542, 0.18483542, 0.18483542],
      dtype=float32))


In [50]:
# Randomly select favorite titles
import random
my_favs = random.sample(list(matching_rows["title_book"]), k=4)
my_favs

['Pies Are Awesome: The Definitive Pie Art Book: Step-by-Step Designs for All Occasions',
 'The Wrong Trail Knife',
 'Palace of Treason: A Novel (2) (The Red Sparrow Trilogy)',
 'More Serious Pleasure: Lesbian Erotic Stories & Poetry']

In [51]:
# Get item ids
my_favs_ids = [title_to_index[f] for f in my_favs]
my_favs_ids

[2630, 1109, 1952, 1500]

In [ ]:
fav_vectors = [model.item_factors[i] for i in my_favs_ids]
fav_vectors

[array([-0.01394593,  0.01170271, -0.00812605,  0.00027586,  0.02342346,
        -0.04799267,  0.02325197,  0.01171427,  0.00456432, -0.00574119,
        -0.03414536,  0.00955969,  0.00185939, -0.00164892,  0.01362613,
         0.02153937,  0.0095694 ,  0.01898364, -0.00355444,  0.00498693,
         0.0316475 ,  0.01122987, -0.00529563,  0.04867561, -0.00955822,
        -0.00542285,  0.00180723, -0.0186466 ,  0.00719062, -0.04284658,
         0.00190123,  0.02184168,  0.01895089,  0.01692701, -0.01466322,
         0.00125175, -0.03016873,  0.01662938, -0.03829423,  0.00597109,
        -0.00309228,  0.02978549, -0.02918064,  0.02439965, -0.03055639,
        -0.00066314,  0.0052548 ,  0.02690804, -0.03297873, -0.02359251],
       dtype=float32),
 array([ 0.00517838,  0.00327216,  0.00268708,  0.00048994,  0.00607853,
         0.00440413,  0.00557926, -0.00220856,  0.00718402,  0.00648132,
         0.00193852,  0.00365416, -0.0042805 ,  0.00230947,  0.00140601,
         0.00757266,  0.007

In [54]:
len(fav_vectors[0])

50

In [55]:
# Average the vectors
avg_vec = np.average(np.stack(fav_vectors), axis=0)
avg_vec

array([-0.00663317,  0.00078971, -0.00314562,  0.00340678,  0.00948072,
       -0.00917555,  0.00608801,  0.00248078,  0.0034182 , -0.00652725,
       -0.00316022,  0.00190497, -0.00131712,  0.00289189,  0.00633166,
        0.00436534, -0.00265375,  0.00658244,  0.00397713,  0.002028  ,
        0.00830958,  0.00494851, -0.00197229,  0.01708262,  0.00125137,
       -0.00257318,  0.0045801 , -0.00169524,  0.00500983, -0.00317971,
        0.00711974,  0.01354905,  0.01089291,  0.0142497 , -0.00301383,
        0.00087   , -0.00863348,  0.00443289, -0.00125182,  0.0050304 ,
       -0.00667061,  0.009253  , -0.00884225,  0.00753987,  0.00131414,
        0.00842462,  0.00775864,  0.01426969, -0.01083482, -0.0039086 ],
      dtype=float32)

In [ ]:
recommendations = np.argsort(np.dot(avg_vec, model.item_factors.T))[:10]
recommendations

array([1607,  735,  961, 2695,  612,  993, 1443, 1615,  321,  528])

In [59]:
# Show original books
my_favs

['Pies Are Awesome: The Definitive Pie Art Book: Step-by-Step Designs for All Occasions',
 'The Wrong Trail Knife',
 'Palace of Treason: A Novel (2) (The Red Sparrow Trilogy)',
 'More Serious Pleasure: Lesbian Erotic Stories & Poetry']

In [61]:
# Show recommendations (titles)
for r in recommendations:
    print(index_to_title[r])

Prettiest Doll
Fancy Nancy - Electronic Me Reader and 8 Sound Book Library - PI Kids
Heirloom: Time-Honored Techniques, Nourishing Traditions, and Modern Recipes
Victory. Stand!: Raising My Fist for Justice
Draw 50 Airplanes, Aircraft, and Spacecraft: The Step-by-Step Way to Draw World War II Fighter Planes, Modern Jets, Space Capsules, and Much More...
Hop on Pop (I Can Read It All By Myself)
Night Flight to Paris: Kate Rees, Book 2
Professional C++
Blood Pressure Log Book: Blood Pressure Chart Cover — Blood Pressure Journal Diary & Heart Rate Pulse Monitor Tracker w/ 104 Weekly Log Sheets (2 ... BP Readings at Home | Minimal Black Design
Dead Men's Harvest (Joe Hunter Novels, 6)


## Using Smaller Dataset from Kaggle

In [3]:
import pandas as pd

In [4]:
# Read dataframes
reviews = pd.read_csv("../../data/raw/Books_rating.csv")
reviews.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [71]:
# Review columns
print(reviews.columns)

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')


In [75]:
# Load in the books dataset. 
books = pd.read_csv("../../data/raw/books_data.csv")
books.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [76]:
print(books.columns)

Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'publishedDate', 'infoLink', 'categories', 'ratingsCount'],
      dtype='object')


In [82]:
# Are the title columns the same?
reviews["Title"].isin(books["Title"]).unique()

array([ True])

In [10]:
# Yes, the titles in the reviews dataset all belong to the books dataset, 
# removing the need to do a merge for just collaborative filtering. 
# We do need to clean the data

reviews.isnull().sum()

User_id         0
Title           0
review/score    0
dtype: int64

In [85]:
# We don't really need the price, profileName, but the User_id is kind of important
# for creating our user-item interaction table. 
561787/len(reviews)

0.18726233333333334

In [5]:
# 18% of the reviews don't have a user associated with them. I think we have to drop these
# rows because they can't be used for collaborative filtering. 
# First, the subset we'll start with for filtering is the User_id, Title, and review/score.
# We can drop any null values for those columns. 
cols = ["User_id", "Title", "review/score"]
reviews = reviews[cols]
reviews.head()

,User_id,Title,review/score
0,AVCGYZL8FQQTD,Its Only Art If Its Well Hung!,4.0
1,A30TK6U7DNS82R,Dr. Seuss: American Icon,5.0
2,A3UH4UZ4RSVO82,Dr. Seuss: American Icon,5.0
3,A2MVUWT453QH61,Dr. Seuss: American Icon,4.0
4,A22X4XUPKF66MR,Dr. Seuss: American Icon,4.0


In [6]:
reviews = reviews.dropna(axis=0)

In [89]:
len(reviews)

2438018

Since Implicit uses the number more as a confidence in the book, rather than a positive or negative indication, the model may perform better if negative reviews were removed. Otherwise, even though someone may have indicated one book as 1 star, the model will still see it as something that the user interacted with and use the 1 as a confidence level, so similar books could still be recommended. Since over half the reviews are 4 and 5 stars, it seems worth using just those in the data. I decided to keep the actual values as 4 and 5, giving the concept of confidence and shifting recommendations more towards 5-star reviews. 

In [12]:
len(reviews[reviews["review/score"] >= 4])

1954329

In [ ]:
reviews = reviews[reviews["review/score"] >= 4]

In [7]:
# Now we can form the sparse user interaction table. 
user_item_matrix = sparse.csr_matrix((reviews['review/score'], (reviews['User_id'].astype('category').cat.codes, reviews['Title'].astype('category').cat.codes)))

NameError: name 'sparse' is not defined

In [92]:
# Initialize the ALS model
model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=20)

# Train the model
model.fit(user_item_matrix)

100%|██████████| 20/20 [05:23<00:00, 16.17s/it]


In [ ]:
# Create the Categorical Series first
title_category = matching_rows["title_book"].astype("category")

title_to_index = dict(zip(title_category.cat.categories, title_category.cat.codes))

index_to_title = dict(enumerate(title_category.cat.categories))

The following code tests the model with a user wiith random favorite books. Note that the user doesn't have to provide ratings for the books because the implicit library builds knowledge around the general consensus of whether a book was liked. However, I might explore the idea of having a user provide star ratings and using them with a weighted average to compute the user vector. 

In [105]:
# Randomly select favorite titles
import random
my_favs = random.sample(list(reviews["Title"]), k=4)
my_favs

['The Franchise Affair (Thorndike Press Large Print Buckinghams)',
 'The Picture of Dorian Gray',
 'Golden Retrievers For Dummies (For Dummies (Computer/Tech))',
 'Original Wild Ones']

In [106]:
# Get item ids
my_favs_ids = [title_to_index[f] for f in my_favs]
my_favs_ids

[174499, 16482, 196843, 67250]

In [107]:
fav_vectors = [model.item_factors[i] for i in my_favs_ids]
fav_vectors

[array([ 0.01502893,  0.03314844,  0.01691284,  0.02169546,  0.03208583,
         0.00347026,  0.04667235,  0.01892682,  0.00184343, -0.00496249,
        -0.00363316,  0.014842  ,  0.00555563,  0.05456043,  0.03284366,
         0.04952963,  0.03161456,  0.03869647,  0.05180746, -0.0154483 ,
         0.02283908,  0.06085859,  0.00132466,  0.03610581, -0.00458186,
         0.0181159 ,  0.01545389,  0.04713809,  0.0441594 ,  0.03201421,
        -0.00397152,  0.03173293, -0.00134741,  0.01091346,  0.03302596,
         0.0437598 ,  0.0255789 ,  0.00162413,  0.01702751,  0.01627751,
         0.02468107,  0.00601538,  0.02623918,  0.0466875 ,  0.02521065,
         0.02493602, -0.00103786, -0.0211031 ,  0.02422773,  0.01401481],
       dtype=float32),
 array([ 0.25793406,  0.19701256,  0.1959509 ,  0.16426979,  0.04862445,
         0.10064099,  0.03601914, -0.15771529, -0.03945087,  0.19947618,
        -0.09569312,  0.11346762,  0.23328371,  0.10098659, -0.03834781,
        -0.12911616,  0.085

In [108]:
len(fav_vectors[0])

50

In [109]:
# Average the vectors
avg_vec = np.average(np.stack(fav_vectors), axis=0)
avg_vec

array([ 0.06850944,  0.05783312,  0.05353564,  0.04666886,  0.02059627,
        0.02625941,  0.02072454, -0.03461343, -0.00930095,  0.04903954,
       -0.02465147,  0.03203323,  0.0599358 ,  0.03902556, -0.00125285,
       -0.01993143,  0.02966344,  0.00684742, -0.0354249 , -0.06776582,
        0.02462431, -0.04801182,  0.00431201,  0.04970264, -0.02438489,
        0.02658612,  0.01916007,  0.0324045 , -0.00818865, -0.0454378 ,
        0.03221193,  0.00848428, -0.02996672,  0.07059262, -0.05153555,
        0.03897408,  0.03640704, -0.01758378,  0.02723745,  0.01034231,
       -0.05671796, -0.02023474,  0.0785521 ,  0.0347832 ,  0.00782296,
       -0.02391158, -0.08896232,  0.01529577,  0.09311966,  0.10041896],
      dtype=float32)

In [110]:
recommendations = np.argsort(np.dot(avg_vec, model.item_factors.T))[:10]
recommendations

array([194417,  47821, 177000, 149457, 154924,  49445,   3142,  92557,
       193921,  92541])

In [111]:
# Show original books
my_favs

['The Franchise Affair (Thorndike Press Large Print Buckinghams)',
 'The Picture of Dorian Gray',
 'Golden Retrievers For Dummies (For Dummies (Computer/Tech))',
 'Original Wild Ones']

In [112]:
# Show recommendations (titles)
for r in recommendations:
    print(index_to_title[r])

Unfit for Command: Swift Boat Veterans Speak Out Against John Kerry
ERAGON: INHERITANCE, BOOK ONE.
The Stranger
The Awakening
The Count of Monte Cristo
Eldest (Inheritance, Book 2)
A Farewell to Arms
Lord of the flies
Under the Banner of Heaven
Lord of the Flies
